In [1]:
from ludwig.api import LudwigModel
from ludwig.visualize import learning_curves
import logging
import shutil
import os
import glob
import yaml
from collections import namedtuple

/Users/adammakarucha/.virtualenvs/ludwig/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/adammakarucha/.virtualenvs/ludwig/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/adammakarucha/.virtualenvs/ludwig/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/adammakarucha/.virtua

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

 matplotlib or seaborn are not installed. In order to install all visualization dependencies run pip install ludwig[viz]


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/adammakarucha/.virtualenvs/ludwig/lib/python3.7/site-packages/ludwig/utils/visualization_utils.py", line 33, in <module>
    import matplotlib as mpl
ModuleNotFoundError: No module named 'matplotlib'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/adammakarucha/.virtualenvs/ludwig/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-b0a9c9077f59>", line 2, in <module>
    from ludwig.visualize import learning_curves
  File "/Users/adammakarucha/.virtualenvs/ludwig/lib/python3.7/site-packages/ludwig/visualize.py", line 35, in <module>
    from ludwig.utils import visualization_utils
  File "/Users/adammakarucha/.virtualenvs/ludwig/lib/python3.7/site-packages/ludwig/utils/visualization_utils.py", line 51, in <module>
    sys.exit(-1)
SystemExit: -1

During h

SystemExit: -1

/Users/adammakarucha/.virtualenvs/ludwig/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:

# # Multiple Model Training Example
# 
# This example trains multiple models and extracts training statistics


In [ ]:
def clean_results():
    try:
        shutil.rmtree('./results')
    except:
        pass

    try:
        shutil.rmtree('./visualizations')
    except:
        pass

    try:
        file_list = glob.glob('./data/*.json')
        file_list.append(glob.glob('./data/*.hdf5'))
        os.remove(file_list)
    except:
        pass

In [ ]:
## Clean up old results folder
clean_results()

In [ ]:
# read in base model definition
with open('./model_definition.yaml', 'r') as f:
    base_model = yaml.safe_load(f.read())



In [ ]:
# Specify named tuple to keep track of training results
TrainingResult = namedtuple('TrainingResult', ['name', 'train_stats'])

# specify alternative architectures to test
FullyConnectedLayers = namedtuple('FullyConnectedLayers',['name', 'fc_layers'])

list_of_fc_layers = [
    FullyConnectedLayers(name='Option1', fc_layers=[{'fc_size':64, 'dropout': 'true'}]),

    FullyConnectedLayers(name='Option2', fc_layers=[{'fc_size':128, 'dropout':'true'},
                                                    {'fc_size':64, 'dropout': 'true'}]),

    FullyConnectedLayers(name='Option3', fc_layers=[{'fc_size':128, 'dropout':'true'}])
]

list_of_train_stats = []


In [ ]:
# ## Train models
for model_option in list_of_fc_layers:
    print('>>>> training: ', model_option.name)

    # set up Python dictionary to hold model training parameters
    model_definition = base_model.copy()
    model_definition['input_features'][0]['fc_layers'] = model_option.fc_layers
    model_definition['training']['epochs'] = 8

    # Define Ludwig model object that drive model training
    model = LudwigModel(model_definition,
                        logging_level=logging.INFO)

    # initiate model training
    train_stats = model.train(data_csv='./data/mnist_dataset_training.csv',
                             experiment_name='multiple_experiment',
                             model_name=model_option.name)

    # save training stats for later use
    list_of_train_stats.append(TrainingResult(name=model_option.name, train_stats=train_stats))

    print('>>>>>>> completed: ', model_option.name, '\n')

    model.close()

In [ ]:
# generating learning curves from training
option_names = [trs.name for trs in list_of_train_stats]
train_stats = [trs.train_stats for trs in list_of_train_stats]
learning_curves(train_stats, 'Survived',
                model_names=option_names,
                output_directory='./visualizations',
                file_format='png')



